# 사용할 함수 정의

In [91]:
import cufflinks as cf 
cf.go_offline(connected=True)
cf.set_config_file(theme='polar')

pd.set_option('display.max_columns', 500)

pd.set_option('display.width', 1000)

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
plt.rcParams["font.family"] = 'Nanum Gothic'

import os
import folium
import pandas as pd
import geopandas as gpd
import numpy as np
import deckgljupyter.Layer as deckgl
import pydeck as pdk

from shapely.geometry import Polygon, Point

#Pydeck 사용을 위한 함수 정의
import shapely # Shapely 형태의 데이터를 받아 내부 좌표들을 List안에 반환합니다. 
def line_string_to_coordinates(line_string): 
    if isinstance(line_string, shapely.geometry.linestring.LineString): 
        lon, lat = line_string.xy 
        return [[x, y] for x, y in zip(lon, lat)] 
    elif isinstance(line_string, shapely.geometry.multilinestring.MultiLineString): 
        ret = [] 
        for i in range(len(line_string)): 
            lon, lat = line_string[i].xy 
            for x, y in zip(lon, lat): 
                ret.append([x, y])
        return ret 
def multipolygon_to_coordinates(x): 
    lon, lat = x[0].exterior.xy 
    return [[x, y] for x, y in zip(lon, lat)] 
def polygon_to_coordinates(x): 
    lon, lat = x.exterior.xy 
    return [[x, y] for x, y in zip(lon, lat)] 
def to_coordinates(x): 
    try: 
        lon, lat = x[0].exterior.xy 
    except:
        lon, lat = x.exterior.xy 
    return [[x, y] for x, y in zip(lon, lat)] 

""" 교통량 추가 (도로 데이터)"""
def add_traffic(df1, df2, time, traffic_type):
    
    #df1['coordinate'] = df1['geometry'].buffer(0.001).apply(polygon_to_coordinates) 
    #df1 = pd.DataFrame(df1) # geopanadas 가 아닌 pandas 의 데이터프레임으로 꼭 바꿔줘야 합니다. 
    #df1['정규화도로폭'] = df1['width'].apply(int) / df1['width'].apply(int).max()
    
    # time(시간적범위), traffic_type(승용차 or 버스 or 화물차) 지정
    df2_time = df2[df2['시간적범위'] == time]
    
    df1_ = []
    for i in df1.link_id:
        df1_.append([i,sum(df2_time[df2_time['상세도로망_LinkID'].apply(str).str.contains(i)][  traffic_type  ])])
                
    df1_ = pd.DataFrame(df1_).fillna(0)
    df1_.columns = ["link_id", traffic_type[:3]+'_'+str(time)+'시']
        
    df_1_2_time = pd.merge(df1, df1_, on = 'link_id' )

    return df_1_2_time

""" 혼잡도 추가 (도로 데이터) """
def add_traffic_jam(df1, df2):
    
    # 혼합빈도강도 양방향 총 합
    df1_ = []
    for i in df1.link_id:
        df1_.append([i,sum(df2[df2['상세도로망_LinkID'].apply(str).str.contains(i)][df2.columns[9]])])

    df1_ = pd.DataFrame(df1_).fillna(0)
    df1_.columns = ["link_id", df2.columns[9]+'합']
    df_1_2 = pd.merge(df1, df1_,on = 'link_id' )

    return df_1_2

""" 교통량or혼잡도 추가 (격자 데이터에 추가)"""

# df1 = df_grid_0, df2 = df_23_combined, traffic_type = 승용차 추정교통량, 혼잡빈도강도합, 혼잡시간강도합

def add_traffic_to_grid(df1, df2, traffic_type): 
    
    """ 중심점 가져오기 """
    point_cent= gpd.GeoDataFrame(df1[['gid','geo_cent']],geometry = 'geo_cent')
    
    """ 도로 데이터에 buffer, coordinate 추가 """
    #coordinate 
    df2['geometry_buff'] = df2['geometry'].buffer(0.001)
    df2['coordinates'] = df2['geometry'].buffer(0.001).apply(to_coordinates)
    # coordinate 
    df2['coordinates_str'] = df2['geometry'].apply(line_string_to_coordinates)
    
    """ superset 생성: 도로 데이터에서 geometry_buff 가져옴 """
    df_superset = df2[df2[traffic_type]>0] #교통량이 0인 도로 제외
    df_superset = gpd.GeoDataFrame(df_superset, geometry = "geometry_buff")
    
    """ grid_ids 생성: 격자의 중심점에 도로의 buffer에 포함되는지 확인 -> 포함되면, 도로 레코드에 gid 추가함 """
    """ 도로 하나씩 loop -> point_cent 중 해당되는 point 있는지 boolean으로 찾기 (.within) 
         -> 해당되는 point의 id를 리스트에 넣기(grid_ids) -> grid_ids를 다시 새로운 리스트에 모으기 (df_grid)"""
    df_grid = []
    for i in range(len(df_superset)): 
        try:
            grid_ids = point_cent[point_cent.within(df_superset.loc[i,'geometry_buff'])]["gid"]
            if len(grid_ids) != 0:
                df_grid.append([i,str(tuple(grid_ids))])
        except :
            pass
    print(traffic_type, ' --- Point와 관련된 grid 개수: ',len(df_grid))
    
    """ 도로 레코드에 grid_ids 부여: 각 도로가 속하는 격자 id들의 리스트 """
    df_superset['grid_ids'] = 0
    for i in range(len(df_grid)):
        id_idx = df_grid[i][0]
        grids = df_grid[i][1]
        df_superset['grid_ids'][id_idx] = grids
    
    """ grid_list 생성: [gid, 교통량]이 item인 형태  ---> 교통량/혼잡도 SUM """
    grid_list = []
    for i in df1['gid']:
        try:
            grid_list.append([i, sum(df_superset[df_superset['grid_ids'].str.contains(i)==True][traffic_type])])
        except:
            pass
    
    """ traffic type 부여 """
    grid_=pd.DataFrame(grid_list)
    grid_.columns = ["gid", traffic_type]
    df_result = pd.merge(df1, grid_, on = 'gid')
    
    return df_result


""" 장소 카운트 위한 함수 정의 """
def location_count(df1, df2, col_name):
    
    df1[col_name] = 0
    
    for i,row in df2.iterrows():
        
        lat = [col for col in list(df2.columns) if '위도' in col][0]
        lon = [col for col in list(df2.columns) if '경도' in col][0]
        
        for index,row2 in df1.iterrows():
            
            if row2.lon_min <= row[lon] <= row2.lon_max and row2.lat_min <= row[lat] <= row2.lat_max:
                df1.at[index, col_name] += 1
                break
    
    #print(df1[col_name].value_counts())
    return df1.sample()

""" 값 카운트 위한 함수 정의 """
def value_count(df1, df2, col_name, val):
    
    df1[col_name] = 0
    df1[col_name] = df1[col_name].map(lambda x: float(x))
    
    for i,row in df2.iterrows():
        
        lat = [col for col in list(df2.columns) if '위도' in col][0]
        lon = [col for col in list(df2.columns) if '경도' in col][0]
        
        for index,row2 in df1.iterrows():
            
            if row2.lon_min <= row[lon] <= row2.lon_max and row2.lat_min <= row[lat] <= row2.lat_max:
                df1.at[index, col_name] += row[val]
                break
    
    #print(df1[col_name].value_counts())
    return df1.sample()

''' 장소 평균 추가 위한 함수 정의 '''
def location_mean(df1, df2, col_name,cnt,val):
    
    df1[col_name] = 0
    df1[cnt] = 0
    df1[col_name] = df1[col_name].map(lambda x:float(x))
    
    for i,row in df2.iterrows():
        
        lat = [col for col in list(df2.columns) if '위도' in col][0]
        lon = [col for col in list(df2.columns) if '경도' in col][0]
        
        for index,row2 in df1.iterrows():
            
            if row2.lon_min <= row[lon] <= row2.lon_max and row2.lat_min <= row[lat] <= row2.lat_max:
                df1.at[index, col_name] += row[val]
                df1.at[index, cnt] += 1

# 데이터 불러오기

In [92]:
from geoband import API
import pathlib

input_path = pathlib.Path('./input')
if not input_path.is_dir():
    input_path.mkdir()
    
API.GetCompasData('SBJ_2012_001', '1', input_path.joinpath('1.오산시_주정차단속(2018~2020).csv'))
API.GetCompasData('SBJ_2012_001', '2', input_path.joinpath('2.오산시_어린이교통사고_격자.geojson'))
API.GetCompasData('SBJ_2012_001', '3', input_path.joinpath('3.오산시_차량등록현황_격자.geojson'))
API.GetCompasData('SBJ_2012_001', '4', input_path.joinpath('4.오산시_연령별_거주인구격자(총인구).geojson'))
API.GetCompasData('SBJ_2012_001', '5', input_path.joinpath('5.오산시_연령별_거주인구격자(유소년).geojson'))
API.GetCompasData('SBJ_2012_001', '6', input_path.joinpath('6.오산시_연령별_거주인구격자(생산가능인구).geojson'))
API.GetCompasData('SBJ_2012_001', '7', input_path.joinpath('7.오산시_연령별_거주인구격자(고령).geojson'))
API.GetCompasData('SBJ_2012_001', '8', input_path.joinpath('8.오산시_유동인구(2019).csv'))
API.GetCompasData('SBJ_2012_001', '9', input_path.joinpath('9.오산시_어린이보호구역.csv'))
API.GetCompasData('SBJ_2012_001', '10', input_path.joinpath('10.오산시_학교위치정보.csv'))
API.GetCompasData('SBJ_2012_001', '11', input_path.joinpath('11.오산시_초등학교_통학구.geojson'))
API.GetCompasData('SBJ_2012_001', '12', input_path.joinpath('12.오산시_중학교_학군.geojson'))
API.GetCompasData('SBJ_2012_001', '13', input_path.joinpath('13.오산시_어린이집_유치원현황.csv'))
API.GetCompasData('SBJ_2012_001', '14', input_path.joinpath('14.오산시_기상데이터(2010~2019).csv'))
API.GetCompasData('SBJ_2012_001', '15', input_path.joinpath('15.오산시_무인교통단속카메라.csv'))
API.GetCompasData('SBJ_2012_001', '16', input_path.joinpath('16.오산시_도로안전표지표준데이터.csv'))
API.GetCompasData('SBJ_2012_001', '17', input_path.joinpath('17.오산시_횡단보도.geojson'))
API.GetCompasData('SBJ_2012_001', '18', input_path.joinpath('18.오산시_과속방지턱표준데이터.csv'))
API.GetCompasData('SBJ_2012_001', '19', input_path.joinpath('19.오산시_신호등.geojson'))
API.GetCompasData('SBJ_2012_001', '20', input_path.joinpath('20.오산시_CCTV설치현황.csv'))
API.GetCompasData('SBJ_2012_001', '21', input_path.joinpath('21.오산시_인도.geojson'))
API.GetCompasData('SBJ_2012_001', '22', input_path.joinpath('22.오산시_버스정류장.csv'))
API.GetCompasData('SBJ_2012_001', '23', input_path.joinpath('23.오산시_상세도로망_LV6.geojson'))
API.GetCompasData('SBJ_2012_001', '24', input_path.joinpath('24.평일_전일,시간대별_오산시_추정교통량_Level6.csv'))
API.GetCompasData('SBJ_2012_001', '25', input_path.joinpath('25.평일_전일_오산시_혼잡빈도강도_Level6.csv'))
API.GetCompasData('SBJ_2012_001', '26', input_path.joinpath('26.평일_전일_오산시_혼잡시간강도_Level6.csv'))
API.GetCompasData('SBJ_2012_001', '27', input_path.joinpath('27.오산시_도로명주소_건물.geojson'))
API.GetCompasData('SBJ_2012_001', '28', input_path.joinpath('28.오산시_건물연면적_격자.geojson'))
API.GetCompasData('SBJ_2012_001', '29', input_path.joinpath('29.오산시_체육시설현황.csv'))
API.GetCompasData('SBJ_2012_001', '30', input_path.joinpath('30.오산시_학원_및_교습소_현황.csv'))
API.GetCompasData('SBJ_2012_001', '31', input_path.joinpath('31.오산시_법정경계(시군구).geojson'))
API.GetCompasData('SBJ_2012_001', '32', input_path.joinpath('32.오산시_행정경계(읍면동).geojson'))
API.GetCompasData('SBJ_2012_001', '33', input_path.joinpath('33.오산시_법정경계(읍면동).geojson'))
API.GetCompasData('SBJ_2012_001', '34', input_path.joinpath('34.오산시_지적도.geojson'))
API.GetCompasData('SBJ_2012_001', '35', input_path.joinpath('35.코드정의서.xlsx'))

for path in list(input_path.glob('*csv')) + list(input_path.glob('*.geojson')):
    print(path)
    
df_01 = pd.read_csv('input/1.오산시_주정차단속(2018~2020).csv')

df_02 = gpd.read_file('input/2.오산시_어린이교통사고_격자.geojson')
df_03 = gpd.read_file('input/3.오산시_차량등록현황_격자.geojson')
df_04 = gpd.read_file('input/4.오산시_연령별_거주인구격자(총인구).geojson')
df_05 = gpd.read_file('input/5.오산시_연령별_거주인구격자(유소년).geojson')
df_06 = gpd.read_file('input/6.오산시_연령별_거주인구격자(생산가능인구).geojson')
df_07 = gpd.read_file('input/7.오산시_연령별_거주인구격자(고령).geojson')

df_08 = pd.read_csv('input/8.오산시_유동인구(2019).csv')
df_09 = pd.read_csv('input/9.오산시_어린이보호구역.csv')
df_10 = pd.read_csv('input/10.오산시_학교위치정보.csv')

df_11 = gpd.read_file('input/11.오산시_초등학교_통학구.geojson')
df_12 = gpd.read_file('input/12.오산시_중학교_학군.geojson')

df_13 = pd.read_csv('input/13.오산시_어린이집_유치원현황.csv')
df_14 = pd.read_csv('input/14.오산시_기상데이터(2010~2019).csv')
df_15 = pd.read_csv('input/15.오산시_무인교통단속카메라.csv')
df_16 = pd.read_csv('input/16.오산시_도로안전표지표준데이터.csv')

df_17 = gpd.read_file('input/17.오산시_횡단보도.geojson')

df_18 = pd.read_csv('input/18.오산시_과속방지턱표준데이터.csv')

df_19 = gpd.read_file('input/19.오산시_신호등.geojson')

df_20 = pd.read_csv('input/20.오산시_CCTV설치현황.csv')

df_21 = gpd.read_file('input/21.오산시_인도.geojson')

df_22 = pd.read_csv('input/22.오산시_버스정류장.csv')

df_23 = gpd.read_file('input/23.오산시_상세도로망_LV6.geojson')

df_24 = pd.read_csv('input/24.평일_전일,시간대별_오산시_추정교통량_Level6.csv')
df_25 = pd.read_csv('input/25.평일_전일_오산시_혼잡빈도강도_Level6.csv')
df_26 = pd.read_csv('input/26.평일_전일_오산시_혼잡시간강도_Level6.csv')

df_27 = gpd.read_file('input/27.오산시_도로명주소_건물.geojson')
df_28 = gpd.read_file('input/28.오산시_건물연면적_격자.geojson')

df_29 = pd.read_csv('input/29.오산시_체육시설현황.csv')
df_30 = pd.read_csv('input/30.오산시_학원_및_교습소_현황.csv')

df_31= gpd.read_file('input/31.오산시_법정경계(시군구).geojson')
df_32 = gpd.read_file('input/32.오산시_행정경계(읍면동).geojson')
df_33 = gpd.read_file('input/33.오산시_법정경계(읍면동).geojson')
df_34 = gpd.read_file('input/34.오산시_지적도.geojson')

df_35 = pd.read_excel('input/35.코드정의서.xlsx')

input/1.오산시_주정차단속(2018~2020).csv
input/8.오산시_유동인구(2019).csv
input/9.오산시_어린이보호구역.csv
input/10.오산시_학교위치정보.csv
input/13.오산시_어린이집_유치원현황.csv
input/14.오산시_기상데이터(2010~2019).csv
input/15.오산시_무인교통단속카메라.csv
input/16.오산시_도로안전표지표준데이터.csv
input/18.오산시_과속방지턱표준데이터.csv
input/20.오산시_CCTV설치현황.csv
input/22.오산시_버스정류장.csv
input/24.평일_전일,시간대별_오산시_추정교통량_Level6.csv
input/25.평일_전일_오산시_혼잡빈도강도_Level6.csv
input/26.평일_전일_오산시_혼잡시간강도_Level6.csv
input/29.오산시_체육시설현황.csv
input/30.오산시_학원_및_교습소_현황.csv
input/2.오산시_어린이교통사고_격자.geojson
input/3.오산시_차량등록현황_격자.geojson
input/4.오산시_연령별_거주인구격자(총인구).geojson
input/5.오산시_연령별_거주인구격자(유소년).geojson
input/6.오산시_연령별_거주인구격자(생산가능인구).geojson
input/7.오산시_연령별_거주인구격자(고령).geojson
input/11.오산시_초등학교_통학구.geojson
input/12.오산시_중학교_학군.geojson
input/17.오산시_횡단보도.geojson
input/19.오산시_신호등.geojson
input/21.오산시_인도.geojson
input/23.오산시_상세도로망_LV6.geojson
input/27.오산시_도로명주소_건물.geojson
input/28.오산시_건물연면적_격자.geojson
input/31.오산시_법정경계(시군구).geojson
input/32.오산시_행정경계(읍면동).geojson
input/33.오산시_법정경계(읍면동).geojson
input

# 목차
- 1. 어린이보호구역 격자 데이터셋 생성
- 2. 효율성 분석(DEA)
- 3. 교통사고 위험지수
- 4. (결과1) 시설물 우선설치지역 선정
- 5. (결과2) 추가설치 시설물 종류 및 개수 파악


# 1. 어린이보호구역 격자 데이터셋 생성
- 설명: 어린이보호구역에 각종 변수를 추가하기 위해, 어린이보호구역 91개에 대해 반경 200m의 격자 생성
- 사용 데이터: df_09 어린이보호구역

In [93]:
df_09_1 = df_09.copy()
df_09_1['geometry'] = None
lon_1m = 1/88.74/1000
lat_1m = 1/109.96/1000 
rng = 200
for i in range(len(df_09_1)):
    f = []
    a = (df_09_1['보호구역_경도'][i]+lon_1m*rng,df_09_1['보호구역_위도'][i]+lat_1m*rng)
    b = (df_09_1['보호구역_경도'][i]+lon_1m*rng,df_09_1['보호구역_위도'][i]-lat_1m*rng)
    c = (df_09_1['보호구역_경도'][i]-lon_1m*rng,df_09_1['보호구역_위도'][i]-lat_1m*rng)
    d = (df_09_1['보호구역_경도'][i]-lon_1m*rng,df_09_1['보호구역_위도'][i]+lat_1m*rng)
    f.append(a)
    f.append(b)
    f.append(c)
    f.append(d)
    #print(f)
    f_poly = Polygon(f)
    #print(f_poly)
    df_09_1['geometry'][i] = f_poly
df_09_1['coordinates'] = None
for i in range(len(df_09_1['geometry'])):
    lon, lat =df_09_1['geometry'][i].exterior.xy
    df_09_1['coordinates'][i] = [[x, y] for x, y in zip(lon, lat)]
df_09_1_list = []
df_09_1_list2 = []
df_09_1_list3 = []
for i in range(len(df_09_1['geometry'])):
    cent = [[df_09_1['geometry'][i].centroid.coords[0][0],df_09_1['geometry'][i].centroid.coords[0][1]]]
    df_09_1_list.append(cent)
    df_09_1_list2.append(Point(cent[0]))
df_09_1['coord_cent'] = 0
df_09_1['geo_cent'] = 0
df_09_1['coord_cent']= pd.DataFrame(df_09_1_list) # pydeck을 위한 coordinate type
df_09_1['geo_cent'] = df_09_1_list2 # geopandas를 위한 geometry type

""" 100X100 Polygone grid의 최대, 최소 위도/경도 컬럼을 추가 (나중에 계산할때 사용) """
lat_min=[];lat_max=[];lon_min=[];lon_max=[]
for i in range(len(df_09_1)):    
    lon_min.append(df_09_1['coordinates'][i][3][0]) # 1번과제에서는 1,3,0,2 썼는데 여기서는 직접 만든 격자라서 3,1,2,0 로 변경
    lon_max.append(df_09_1['coordinates'][i][1][0])
    lat_min.append(df_09_1['coordinates'][i][2][1])
    lat_max.append(df_09_1['coordinates'][i][0][1])
df_09_1['lon_min'] = lon_min;df_09_1['lon_max'] = lon_max;df_09_1['lat_min'] = lat_min;df_09_1['lat_max'] = lat_max

df_grid = df_09_1.copy()
df_grid.rename(columns = {'시설명' : 'gid'}, inplace = True)
df_grid.drop(['시설종류', 'CCTV설치여부', '보호구역도로폭'], axis=1, inplace=True)

print(df_grid.shape)
df_grid.sample()

(91, 12)


,gid,CCTV설치대수,보호구역_경도,보호구역_위도,geometry,coordinates,coord_cent,geo_cent,lon_min,lon_max,lat_min,lat_max
25,세마초등학교,3,127.027581,37.16968,"POLYGON ((127.0298352750732 37.17149904321572,...","[[127.02983527507324, 37.17149904321572], [127...","[127.02758150000001, 37.169680199999995]",POINT (127.0275815 37.16968019999999),127.025328,127.029835,37.167861,37.171499


# 2. 효율성 분석(DEA)
- 설명: 효율성 점수를 계산하기 위해, 위에서 만든 격자 데이터셋에 인풋변수, 아웃풋변수를 추가한다

## 2-1. 변수 추가
- 인풋 변수: df_03 차량등록수, df_05유소년인구, df_04총인구, df_24 교통량, df_25 혼잡빈도강도, df_26 혼잡시간강도, df_01 주정차단속, df_08 유동인구, df_10 학교, df_13 유치원, df_30 학원, df_29 체육도장
- 아웃풋 변수 (교통안전시설물): df_09 CCTV(어린이보호구역), df_15 무인교통단속카메라, df_16 도로안전표지, df_20 CCTV(과속방지)

### 2-1-1. 인풋 변수 추가 (코드실행시간: 약12분)

In [94]:
""" 인풋 변수 - 차량등록수,유소년인구,총인구 추가 """

# 결측치 0으로 대체
df_04['val'] = df_04['val'].fillna(0)
df_05['val'] = df_05['val'].fillna(0)
df_grid1 = pd.merge(df_03,df_04.iloc[:,0:2],how='inner',on='gid')
df_grid1.rename(columns=lambda x: x.replace('car_cnt', '차량등록수'), inplace=True)
df_grid1.rename(columns=lambda x: x.replace('val', '총인구'), inplace=True)
df_grid1 = pd.merge(df_grid1,df_05.iloc[:,0:2],how='inner',on='gid')
df_grid1.rename(columns=lambda x: x.replace('val', '유소년인구'), inplace=True)
df_grid1['coordinates'] = df_grid1['geometry'].apply(multipolygon_to_coordinates) #pydeck 을 위한 coordinate type
df_grid1_list = []
df_grid1_list2 = []
for i in df_grid1['geometry']:
    cent = [[i[0].centroid.coords[0][0],i[0].centroid.coords[0][1]]]
    df_grid1_list.append(cent)
    df_grid1_list2.append(Point(cent[0]))
df_grid1['coord_cent'] = 0
df_grid1['geo_cent'] = 0
df_grid1['coord_cent']= pd.DataFrame(df_grid1_list) # pydeck을 위한 coordinate type
df_grid1['geo_cent'] = df_grid1_list2 # geopandas를 위한 geometry type
lat_min=[]
lat_max=[]
lon_min=[]
lon_max=[]
for i in range(len(df_grid1)):    
    lon_min.append(df_grid1['coordinates'][i][1][0])
    lon_max.append(df_grid1['coordinates'][i][3][0])
    lat_min.append(df_grid1['coordinates'][i][0][1])
    lat_max.append(df_grid1['coordinates'][i][2][1])
df_grid1['lon_min'] = lon_min
df_grid1['lon_max'] = lon_max
df_grid1['lat_min'] = lat_min
df_grid1['lat_max'] = lat_max
lon_cent=[]
lat_cent=[]
for i in range(len(df_grid1)):
    lon_cent.append(df_grid1['geometry'][i].centroid.coords[0][0])
    lat_cent.append(df_grid1['geometry'][i].centroid.coords[0][1])
df_grid1['중심경도'] = lon_cent
df_grid1['중심위도'] = lat_cent

location_count(df_grid1,df_09,'어린이보호구역')
df_grid2 = df_grid1[df_grid1['어린이보호구역']>0]

location_mean(df_grid,df_grid2,'차량등록수_sum','차량등록수_cnt','차량등록수')
df_grid['차량등록수'] = df_grid['차량등록수_sum']/df_grid['차량등록수_cnt']

location_mean(df_grid,df_grid2,'유소년인구_sum','유소년인구_cnt','유소년인구')
df_grid['유소년인구'] = df_grid['유소년인구_sum']/df_grid['유소년인구_cnt']

location_mean(df_grid,df_grid2,'총인구_sum','총인구_cnt','총인구')
df_grid['총인구'] = df_grid['총인구_sum']/df_grid['총인구_cnt']

df_grid = df_grid.drop(["차량등록수_sum", "차량등록수_cnt",'유소년인구_sum','유소년인구_cnt','총인구_sum','총인구_cnt'], axis=1)


""" 인풋 변수 - 교통량, 혼잡도 추가 (3분) """

# 24.교통량 데이터 전처리: ['시간적범위'] 값에 str, int 섞여있음 ---> int로 변환 
df_24.시간적범위 = df_24.시간적범위.map(lambda x: int(x) if x != '전일' else '전일')
#  23.상세도로망 데이터에 교통량 column 추가 (오후 2~7시) 
df_23_combined = add_traffic(df_23, df_24, 14, '승용차 추정교통량')
df_23_combined = add_traffic(df_23_combined, df_24, 15, '승용차 추정교통량')
df_23_combined = add_traffic(df_23_combined, df_24, 16, '승용차 추정교통량')
df_23_combined = add_traffic(df_23_combined, df_24, 17, '승용차 추정교통량')
df_23_combined = add_traffic(df_23_combined, df_24, 18, '승용차 추정교통량')
df_23_combined = add_traffic(df_23_combined, df_24, 19, '승용차 추정교통량')
# 14~19시 모두 합친 칼럼 생성
df_23_combined['승용차_14시_19시'] = df_23_combined['승용차_14시']+df_23_combined['승용차_15시']+df_23_combined['승용차_16시']+df_23_combined['승용차_17시']+df_23_combined['승용차_18시']+df_23_combined['승용차_19시']
df_23_combined = df_23_combined.drop(df_23_combined.loc[:, '승용차_14시':'승용차_19시'].columns, axis = 1)
# 23.상세도로망 데이터에 혼잡도 column 추가 
df_23_combined = add_traffic_jam(df_23_combined, df_25)
df_23_combined = add_traffic_jam(df_23_combined, df_26)
# 격자 데이터에 추가 
df_grid = add_traffic_to_grid(df_grid, df_23_combined, '승용차_14시_19시')
df_grid = add_traffic_to_grid(df_grid, df_23_combined, '혼잡빈도강도합')
df_grid = add_traffic_to_grid(df_grid, df_23_combined, '혼잡시간강도합')


""" 인풋 변수 - 주정차단속 추가 (4분) """

print(df_01.shape)
# 전처리 
df_01['오산시_yn'] = df_01['행정구역'].str.slice(start = 4,stop = 7)
df_01 = df_01[df_01['오산시_yn'] == '오산시']
df_01 = df_01.drop(['오산시_yn'],axis = 1)
print(df_01.shape)
# 일자 선택: 2019년 3~6월, 9~12월 (2019년 한 이유는 데이터가 2018-11 ~ 2020-11 이라서)
df_01['단속일자'] = df_01['단속일자'].map(lambda x: str(x))
df_01['단속일자_yr'] = df_01['단속일자'].str.slice(start = 0,stop = 4)
df_01['단속일자_mth'] = df_01['단속일자'].str.slice(start = 4,stop = 6)
df_01['단속일자_yr'] = df_01['단속일자_yr'].map(lambda x: int(x))
df_01['단속일자_mth'] = df_01['단속일자_mth'].map(lambda x: int(x))
year_list = [2019]
month_list = [3,4,5,6,9,10,11,12]
df_01_time = df_01.loc[df_01['단속일자_yr'].isin(year_list)]
df_01_time = df_01_time.loc[df_01_time['단속일자_mth'].isin(month_list)]
print(df_01_time.shape)

location_count(df_grid, df_01_time, '주정차단속')


""" 인풋 변수 - 유동인구 추가 (2분) """

# 3~6월, 9~12월, 9시, 14~19시 사용
months = [201903, 201904, 201905, 201906, 201909, 201910, 201911, 201912]
df_08_time = df_08[df_08.STD_YM.isin(months)]
df_08_time.drop(df_08_time.loc[:, 'TMST_00':'TMST_08'].columns, axis = 1, inplace=True) 
df_08_time.drop(df_08_time.loc[:, 'TMST_10':'TMST_13'].columns, axis = 1, inplace=True) 
df_08_time.drop(df_08_time.loc[:, 'TMST_20':'TMST_23'].columns, axis = 1, inplace=True) 
print(df_08_time.shape)
# 좌표 기준으로 합치기 
df_08_time['TMST_09_14_19'] = df_08_time.apply(lambda x: x['TMST_09']+x['TMST_14']+x['TMST_15']+x['TMST_16']+x['TMST_17']+x['TMST_18']+x['TMST_19'], axis=1)
df_08_time['coordinate'] = df_08_time.apply(lambda x: [x['lon'], x['lat']], axis=1)
df_08_time['coordinate_sum'] = df_08_time.apply(lambda x: x['lon']+ x['lat'], axis=1)
df_08_time.drop(df_08_time.loc[:, 'lon':'TMST_19'].columns, axis = 1, inplace=True) 
# 좌표 별 유동인구 합 구하기 
grouped = df_08_time.groupby(['coordinate_sum'])['TMST_09_14_19']
grouped_sum = pd.DataFrame(grouped.sum())
grouped_sum.rename(columns = {'TMST_09_14_19' : '유동인구'}, inplace = True)
grouped_sum.reset_index(inplace=True)
# 두 데이터프레임 합치기 
df_08_time = df_08_time.drop_duplicates(subset=['coordinate_sum'])
print(df_08_time.shape)
df_sum = grouped_sum.merge(df_08_time, on='coordinate_sum')
df_sum.drop(df_sum.loc[:, 'STD_YM':'TMST_09_14_19'].columns, axis = 1, inplace=True) 
# 격자 데이터에 적용해 필터링: 유동인구 좌표를 location_count해서 집어넣기 
lon_list = []
for i in df_sum.coordinate:
    lon_list.append(i[0])
lat_list = []
for i in df_sum.coordinate:
    lat_list.append(i[1])
df_sum['cent_위도']=lat_list
df_sum['cent_경도']=lon_list

value_count(df_grid, df_sum, '유동인구', '유동인구') 

""" 인풋 변수 - 학교, 어린이집, 유치원  추가 (2분) """

location_count(df_grid, df_10, '초등학교')
location_count(df_grid, df_13, '유치원어린이집')

df_grid['학교_유치원'] = df_grid['초등학교']+df_grid['유치원어린이집']
df_grid = df_grid.drop(['초등학교', '유치원어린이집'], axis=1)


""" 인풋 변수 - 학원, 체육도장 추가 """

df_29 = df_29[ df_29.시설구분명 == '체육도장업' ]
location_count(df_grid, df_29, '체육도장')

df_30 = df_30[(df_30.업종구분명 == '학교교과교습학원') | (df_30.업종구분명 == '교습소')]
location_count(df_grid, df_30, '학원')

df_grid['학원_체육도장'] = df_grid['학원']+df_grid['체육도장']
df_grid = df_grid.drop(['학원', '체육도장'], axis=1)


""" 인풋 변수 - 유동인구 추가 (14시~19시) """

df_08_mean = df_08.astype('float')
df_08_mean['유동인구_14시_19시'] = df_08_mean[['TMST_14','TMST_15','TMST_16','TMST_17','TMST_18','TMST_19']].mean(axis=1)
df_08_mean['유동인구_14시_19시'] = df_08_mean['유동인구_14시_19시'].apply(lambda x: round(x,3))
df_08_mean = df_08_mean[['lon','lat','유동인구_14시_19시']]
df_08_mean = df_08_mean.groupby(['lon', 'lat'], as_index=False).mean()

def location_val(df1, df2, col_name):
    
    df1[col_name] = 0
    
    for index,row2 in df1.iterrows():
        for i,row in df2.iterrows():
        
            if row2.lon_min <= row['lon'] <= row2.lon_max and row2.lat_min <= row['lat'] <= row2.lat_max:
                df1.at[index, col_name] = df2.loc[i,col_name]
                #print(df2.loc[i,col_name])
                break

    #print(df1[col_name].value_counts())
    return df1.sample()

location_val(df_grid, df_08_mean,'유동인구_14시_19시')


print(df_grid.shape)
df_grid.sample()


승용차_14시_19시  --- Point와 관련된 grid 개수:  274
혼잡빈도강도합  --- Point와 관련된 grid 개수:  251
혼잡시간강도합  --- Point와 관련된 grid 개수:  251
(61701, 7)
(61033, 7)
(22455, 9)
(83796, 10)
(12326, 4)
(91, 23)


,gid,CCTV설치대수,보호구역_경도,보호구역_위도,geometry,coordinates,coord_cent,geo_cent,lon_min,lon_max,lat_min,lat_max,차량등록수,유소년인구,총인구,승용차_14시_19시,혼잡빈도강도합,혼잡시간강도합,주정차단속,유동인구,학교_유치원,학원_체육도장,유동인구_14시_19시
17,원동초등학교,2,127.069748,37.135528,"POLYGON ((127.0720017750732 37.13734704321572,...","[[127.07200177507325, 37.13734704321572], [127...","[127.069748, 37.135528199999996]",POINT (127.069748 37.1355282),127.067494,127.072002,37.133709,37.137347,1.0,0.0,0.0,37644.7,1284.36,1591.19,77,51181.97,12,135,3


### 2-1-2. 아웃풋 변수 추가 

In [95]:
""" 아웃풋 변수 - 과속방지턱 추가 """

location_count(df_grid, df_18, '과속방지턱')

""" 아웃풋 변수 - 무인교통단속카메라 추가 """

location_count(df_grid, df_15, '단속카메라_불법주정차')

""" 아웃풋 변수 - 도로안전표지 추가 """

df_16.rename(columns = {'lon' : '경도', 'lat' : '위도'}, inplace = True)
location_count(df_grid, df_16, '도로안전표지')

""" 아웃풋 변수 - 신호등 추가 """

df_19
lon_list = []
for i in df_19.geometry:
    lon_list.append(i.x)
lat_list = []
for i in df_19.geometry:
    lat_list.append(i.y)
df_19['cent_위도']=lat_list
df_19['cent_경도']=lon_list

location_count(df_grid, df_19, '신호등')

""" 아웃풋 변수 - cctv (과속방지) 추가 """

df_20 = df_20[ df_20['CCTV 유형코드']=='E' ]
location_count(df_grid, df_20, 'CCTV_과속단속')

print(df_grid.shape)
df_grid.sample()

(91, 28)


,gid,CCTV설치대수,보호구역_경도,보호구역_위도,geometry,coordinates,coord_cent,geo_cent,lon_min,lon_max,lat_min,lat_max,차량등록수,유소년인구,총인구,승용차_14시_19시,혼잡빈도강도합,혼잡시간강도합,주정차단속,유동인구,학교_유치원,학원_체육도장,유동인구_14시_19시,과속방지턱,단속카메라_불법주정차,도로안전표지,신호등,CCTV_과속단속
10,오산고현초등학교,2,127.085145,37.133417,"POLYGON ((127.0873983750732 37.13523564321571,...","[[127.08739837507325, 37.13523564321571], [127...","[127.08514460000002, 37.133416799999985]",POINT (127.0851446 37.13341679999998),127.082891,127.087398,37.131598,37.135236,38.0,3.5,31.5,21549.22,846.65,904.5,8,23264.5,3,16,4,0,0,4,8,0


In [96]:
""" column 정리: DEA분석에 사용할 수 있는 형태로 만들기 위해 geometry 정보 삭제, 변수 위치 정리 """

dea_data = df_grid.copy()

dea_data.drop(dea_data.loc[:, '보호구역_경도':'lat_max'].columns, axis = 1, inplace=True) 
dea_data.drop(['유동인구_14시_19시'], axis=1, inplace=True)
dea_data.rename(columns = {'CCTV설치대수' : 'CCTV_보호구역'}, inplace = True)
dea_data.rename(columns = {'gid' : '보호구역명'}, inplace = True)

dea_data = dea_data[['보호구역명', '주정차단속', '차량등록수', '유소년인구', '총인구',
                  '유동인구', '학교_유치원', '승용차_14시_19시', '혼잡빈도강도합',
                  '혼잡시간강도합', '학원_체육도장', 'CCTV_보호구역', '단속카메라_불법주정차',
                  '도로안전표지', '과속방지턱', '신호등', 'CCTV_과속단속']]

""" 변수명 수정 (x: 인풋 변수, y: 아웃풋 변수) """

dea_data.columns=['보호구역명', 'x1_주정차단속', 'x2_차량등록수', 'x3_유소년인구',
                                  'x4_총인구','x5_유동인구','x6_학교_유치원','x7_교통량_승용차',
                                 'x8_혼잡빈도강도', 'x9_혼잡시간강도','x10_학원_체육도장',
                                  'y1_CCTV_보호구역', 'y2_단속카메라_불법주정차', 'y3_도로안전표지',
                        'y4_과속방지턱', 'y5_신호등', 'y6_CCTV_과속단속']



print(dea_data.shape)
dea_data.sample()

(91, 17)


,보호구역명,x1_주정차단속,x2_차량등록수,x3_유소년인구,x4_총인구,x5_유동인구,x6_학교_유치원,x7_교통량_승용차,x8_혼잡빈도강도,x9_혼잡시간강도,x10_학원_체육도장,y1_CCTV_보호구역,y2_단속카메라_불법주정차,y3_도로안전표지,y4_과속방지턱,y5_신호등,y6_CCTV_과속단속
45,베스트어린이집,0,6.0,0.0,8.5,4218.14,0,0.0,0.0,0.0,0,1,0,0,0,2,0


In [97]:
dea_data.to_csv('dea_data.csv')

- 효율성 분석을 위해 91곳의 어린이보호구역 격자를 대상으로 인풋변수(x) 10개와 아웃풋변수 6개(y)를 추가하였음
- 위 데이터(dea_data.csv)를 저장 후 R로 옮겨가서 효율성 점수 계산 (y5_신호등 은 제외하고 계산하였으므로 총 5개의 아웃풋변수 사용)

## 2-2. 효율성 점수 계산 (R에서 실행 -> Python으로 결과파일 불러오기)
- 설명: 91개 격자를 효율성을 기준으로 순위를 매긴다
- 2-1. 에서 만든 데이터셋을 가지고 R에서 실행 (R의 'benchmarking' 패키지로 계산) 후, 결과파일(dea_eff.csv, dea_crs.csv)을 가져 옴
- 효율성 점수는 높을수록 비효율적임을 의미 (산출지향 DEA모델을 사용했기 때문 - PPT에 설명)

In [98]:
""" R로 실행한 DEA 결과물 파일 불러오기 """

import pandas as pd
dea_eff = pd.read_csv("dea_eff.csv", index_col=[0]) # 효율성 점수
dea_lambda_crs = pd.read_csv("dea_crs.csv", index_col=[0]) # 벤치마킹을 위한 참조집단 가중치

""" 전처리: column이름에 있는 "." -> " " 변환  """
col_list = list(dea_lambda_crs.columns)
new_list = []
for col in col_list:
    temp = col.replace(".", " ")
    new_list.append(temp)
str(new_list)

"['schoolzone', '성심학교', '가수초등학교', '광성초등학교', '삼미초등학교', '성산초등학교', '성호초등학교', '오산초등학교', '오산원당초등학교', '운암초등학교', '금암초등학교', '양산초등학교', '필봉초등학교', '세미초등학교', '다온초등학교', '세마초등학교', '노틀담유치원', '오산대유치원', '펀키즈 유치원', '뿌리 유치원', '숲속다원유치원', '오르다유치원', '예인나라어린이집', '베스트어린이집', '새싹뜰어린이집', '시립가온어린이집', '청원어린이집', '시립나눔이보듬이어린이집', '하얀뜰어린이집', '키즈청호어린이집', '베이비캐슬어린이집', '좋은아이어린이집', '동부어린이집', '둥근해어린이집', '나리어린이집', '경희어린이집', '대일어린이집', '리틀램어린이집', '아이사랑어린이집', '중앙어린이집', '늘해랑어린이집', '키즈맘 어린이집', '행복아이 어린이집', '자람터어린이집', '시립세마어린이집', '설리반어린이집', '세교복지타운어린이집']"

In [99]:
dea_lambda_crs.columns = ['schoolzone', '성심학교', '가수초등학교', '광성초등학교', '삼미초등학교', '성산초등학교', '성호초등학교', '오산초등학교', '오산원당초등학교', '운암초등학교', '금암초등학교', '양산초등학교', '필봉초등학교', '세미초등학교', '다온초등학교', '세마초등학교', '노틀담유치원', '오산대유치원', '펀키즈 유치원', '뿌리 유치원', '숲속다원유치원', '오르다유치원', '예인나라어린이집', '베스트어린이집', '새싹뜰어린이집', '시립가온어린이집', '청원어린이집', '시립나눔이보듬이어린이집', '하얀뜰어린이집', '키즈청호어린이집', '베이비캐슬어린이집', '좋은아이어린이집', '동부어린이집', '둥근해어린이집', '나리어린이집', '경희어린이집', '대일어린이집', '리틀램어린이집', '아이사랑어린이집', '중앙어린이집', '늘해랑어린이집', '키즈맘 어린이집', '행복아이 어린이집', '자람터어린이집', '시립세마어린이집', '설리반어린이집', '세교복지타운어린이집']

In [100]:
""" 효율성 점수 """

dea_eff.head()

""" crs 비효율 DMU 갯수: 45개 """
dea_eff_crs = dea_eff[dea_eff['crs_eff'] > 1]
dea_eff_crs = dea_eff_crs.sort_values('crs_eff', ascending=False)
print('비효율 DMU 개수: ',len(dea_eff_crs))

""" crs 비효율 상위 41개 저장 (이후 [4.최종선정] 에서 [3.교통사고 위험지수] 상위 40개와 교집합 구할 때 사용) """
dea_eff_41 = dea_eff_crs[:41]
dea_eff_41.head()

비효율 DMU 개수:  45


,schoolzone,crs_eff,vrs_eff
88,미래숲어린이집,6.137375,1
14,오산원일초등학교,4.344148,1
12,오산대원초등학교,3.761123,1
31,오산유치원,3.554243,1
74,아이마루어린이집,3.370313,1


#### (참고) 결과검증용: 비효율 지역(미래숲어린이집) vs 효율 지역(청원어린이집) 비교
- DEA 결과 비효율 지역으로 나타난 미래숲어린이집과 효율 지역으로 나타난 청원어린이집을 비교했을 때, 투입량(x)은 비슷하나 산출량(y, 시설물)에서 큰 차이가 남
- 즉, 비효율지역에서는 투입 요소에 비해 시설물이 부족함

In [101]:
""" 분석 이후 결과의 유의성을 검증하기 위한 용도로 나중에 사용 (PPT 분석결과 부분에 첨부) """

temp_list = ['미래숲어린이집', '청원어린이집']
temp = dea_data.copy()
temp = temp.loc[temp['보호구역명'].isin(temp_list)]
temp.drop(['y5_신호등'], axis = 1, inplace=True)
temp

,보호구역명,x1_주정차단속,x2_차량등록수,x3_유소년인구,x4_총인구,x5_유동인구,x6_학교_유치원,x7_교통량_승용차,x8_혼잡빈도강도,x9_혼잡시간강도,x10_학원_체육도장,y1_CCTV_보호구역,y2_단속카메라_불법주정차,y3_도로안전표지,y4_과속방지턱,y6_CCTV_과속단속
52,청원어린이집,148,103.0,22.0,207.0,11455.45,1,427.21,167.47,192.57,0,1,1,6,0,0
87,미래숲어린이집,101,33.0,87.0,388.0,8128.25,5,613.92,89.42,167.84,17,1,0,0,0,0


# 3. 교통사고 위험지수 
- 설명: 과제 1번에서 개발한 위험지수를 어린이보호구역 격자에도 적용시켜, 위험도 순위를 매긴 후 효율성점수와 함께 고려하여 우선설치 지역을 골라낸다

## 3-1. 변수 추가
- 위험지수 계산에 필요한 변수: 유소년인구, 승용차, 혼잡빈도강도합, 유동인구, 주정차단속, 학원, 아파트, 사거리
- 위에서 이미 만들어 새로 추가할 필요가 없는 변수: 유소년인구, 승용차, 혼잡빈도강도합, 유동인구, 주정차단속
- 새로 추가할 변수: 학원, 아파트, 사거리

In [102]:
print(df_grid.shape)
df_grid.sample()

(91, 28)


,gid,CCTV설치대수,보호구역_경도,보호구역_위도,geometry,coordinates,coord_cent,geo_cent,lon_min,lon_max,lat_min,lat_max,차량등록수,유소년인구,총인구,승용차_14시_19시,혼잡빈도강도합,혼잡시간강도합,주정차단속,유동인구,학교_유치원,학원_체육도장,유동인구_14시_19시,과속방지턱,단속카메라_불법주정차,도로안전표지,신호등,CCTV_과속단속
18,화성초등학교,3,127.069192,37.164773,"POLYGON ((127.0714459750732 37.16659214321571,...","[[127.07144597507325, 37.16659214321571], [127...","[127.0691922, 37.16477329999999]",POINT (127.0691922 37.16477329999999),127.066938,127.071446,37.162954,37.166592,246.0,89.0,612.0,2928.02,878.0,978.01,51,13643.45,3,11,11,0,2,6,30,0


In [103]:
""" 학원 추가 """

df_30 = df_30[(df_30.업종구분명 == '학교교과교습학원') | (df_30.업종구분명 == '교습소')]
location_count(df_grid, df_30, '학원')

""" 아파트 추가 """

# build_cd : 건물용도코드
# 단독주택, 다중주택, 공관
build_cd = df_35.loc[521:849]
build_cd.columns = ['code','codem']
build_cd = build_cd.reset_index(drop = True)
#코드정의서와 건물파일 결합
df_27_k = pd.merge(df_27,build_cd, how='left', left_on='BDTYP_CD', right_on='code')
#코드정의서에서 없는 코드 지우기
index_n = df_27_k[df_27_k['BDTYP_CD'] == '08003'].index
df_27_k = df_27_k.drop(index_n)
#코드정의서의 code지우기
df_27_k = df_27_k.drop('code',axis =1)
df_27_1 = df_27_k.copy()
df_27_1['coordinate'] = df_27_1['geometry'].apply(multipolygon_to_coordinates) 
df_27_1 = pd.DataFrame(df_27_1)
#100X100 grid에서 central point 찾기
df_27_1_list = []
df_27_1_list2 = []
for i in df_27_1['geometry']:
    cent = [[i[0].centroid.coords[0][0],i[0].centroid.coords[0][1]]]
    df_27_1_list.append(cent)
    df_27_1_list2.append(Point(cent[0]))
df_27_1['coord_cent'] = 0
df_27_1['geo_cent'] = 0
df_27_1['coord_cent']= pd.DataFrame(df_27_1_list) # pydeck을 위한 coordinate type
df_27_1['geo_cent'] = df_27_1_list2 # geopandas를 위한 geometry type
#결측값 나오는 부분있어서 안돌아가서 결측값 삭제
df_27_1 = df_27_1[df_27_1['coord_cent'].isnull() == False]
df_27_1 = df_27_1.reset_index()
df_27_1 = df_27_1.drop(['index'],axis = 1)
#중심점 위도,경도 떼어오기
df_27_1["중심경도"] = None
df_27_1["중심위도"] = None
for i in df_27_1['coord_cent'].index:
    df_27_1["중심경도"][i] = df_27_1['coord_cent'][i][0]
    df_27_1["중심위도"][i] = df_27_1['coord_cent'][i][1]
apt = df_27_1[df_27_1['codem'] == '아파트']
apt = apt.reset_index()
apt = apt.drop(['index'],axis = 1)

location_count(df_grid,apt,'아파트수')

""" 사거리 추가 """

df_16 = df_16[df_16.표지종별=='3방향표지']
df_16.rename(columns = {'lon' : '경도', 'lat':'위도'}, inplace = True)
location_count(df_grid, df_16, '사거리')
# 사거리 변수 0,1 로 바꾸기 
df_grid['사거리'] = df_grid['사거리'].apply(lambda x: 1 if x!=0 else 0)

print(df_grid.shape)
df_grid.sample()

(91, 31)


,gid,CCTV설치대수,보호구역_경도,보호구역_위도,geometry,coordinates,coord_cent,geo_cent,lon_min,lon_max,lat_min,lat_max,차량등록수,유소년인구,총인구,승용차_14시_19시,혼잡빈도강도합,혼잡시간강도합,주정차단속,유동인구,학교_유치원,학원_체육도장,유동인구_14시_19시,과속방지턱,단속카메라_불법주정차,도로안전표지,신호등,CCTV_과속단속,학원,아파트수,사거리
36,창의나라 유치원,1,127.049462,37.171456,"POLYGON ((127.0517158750732 37.17327484321572,...","[[127.05171587507324, 37.173274843215715], [12...","[127.04946210000001, 37.17145599999999]",POINT (127.0494621 37.17145599999999),127.047208,127.051716,37.169637,37.173275,27.5,7.5,53.5,5552.11,2513.26,2690.41,0,5284.55,2,0,0,0,0,2,0,0,0,1,0


## 3-2. 위험지수 계산
- 설명: 과제 1번에서 만든 로지스틱 회귀모델의 계수를 활용하여 위험지수를 계산한다

In [104]:
""" column 정리: 교통사고 위험지수 계산에 필요한 변수만 남기고 제거 """

score_data = df_grid.copy()

score_data = score_data.drop(score_data.loc[:, 'CCTV설치대수':'총인구'].columns, axis = 1)
score_data.drop(['혼잡시간강도합'], axis=1, inplace=True)
score_data = score_data.drop(score_data.loc[:, '유동인구':'학원_체육도장'].columns, axis = 1)
score_data = score_data.drop(score_data.loc[:, '과속방지턱':'학원'].columns, axis = 1)


score_data = score_data[['gid', '혼잡빈도강도합', '유동인구_14시_19시', '주정차단속', '아파트수', '승용차_14시_19시', '사거리' ]]

print(score_data.shape)
score_data.sample()

(91, 7)


,gid,혼잡빈도강도합,유동인구_14시_19시,주정차단속,아파트수,승용차_14시_19시,사거리
63,동부어린이집,0.0,0,0,0,0.0,0


In [105]:
""" 
1번과제에서 구한 로지스틱 회귀모델 beta값
순서: 혼잡빈도강도합, 유동인구_14시_19시, 주정차단속, 아파트수, 승용차_14시_19시, 사거리
"""

beta_list = [0.839980, 0.656979, 0.557703, 0.531020, 0.248176, 0.196911]

In [106]:
""" 정규화 """

x_values = score_data.drop(columns=['gid'], axis=1, inplace=False)
col_names = x_values.columns

import pandas as pd
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()

x_scaled = min_max_scaler.fit_transform(x_values.values)
normalized_df = pd.DataFrame(x_scaled, columns = col_names)
data_norm = normalized_df[:]

""" score 추가: 교통사고 위험지수 """

data_norm['score'] = 0
data_norm['score'] = data_norm['score'].map(lambda x: float(x))
for index,row in data_norm.iterrows():
    
    temp = list(row)
    temp.pop()
    temp = np.array(temp) # x변수 array
    
    beta = np.array(beta_list) # beta값 array
    
    score = sum(temp*beta)
    
    data_norm.at[index, 'score'] += score
        
gid_list = list(score_data.gid)
data_norm['gid'] = gid_list

print(data_norm.shape)
data_norm.sample()

(91, 8)


,혼잡빈도강도합,유동인구_14시_19시,주정차단속,아파트수,승용차_14시_19시,사거리,score,gid
6,0.0,0.037037,0.078089,0.016393,0.0,0.0,0.076588,성산초등학교


In [107]:
""" 위험지수 높은 순 정렬 """

data_norm_sorted = data_norm.sort_values('score', ascending=False)
data_norm_sorted.head()

,혼잡빈도강도합,유동인구_14시_19시,주정차단속,아파트수,승용차_14시_19시,사거리,score,gid
88,0.250536,0.351852,1.000000,0.245902,0.340902,1.0,1.411401,시립은여울어린이집
11,0.372247,0.148148,0.123543,0.754098,0.537862,0.0,1.012836,오산대원초등학교
34,1.000000,0.000000,0.001166,0.147541,0.310363,0.0,0.996002,동화마을유치원
48,0.699378,0.074074,0.053030,0.098361,0.232698,1.0,0.972596,시립매홀어린이집
90,0.000000,0.518519,0.590909,0.147541,0.014843,1.0,0.949149,세교복지타운어린이집


In [108]:
""" 위험지수 상위 40곳: 효율성점수 상위 41곳과 교집합 구하기 위해 필요 """

str(list(data_norm_sorted.gid[:40]))

"['시립은여울어린이집', '오산대원초등학교', '동화마을유치원', '시립매홀어린이집', '세교복지타운어린이집', '원동초등학교', '시립세마어린이집', '물향기 유치원', '운암초등학교', '금암초등학교', '오산원일초등학교', '운천초등학교', '성호초등학교', '운산초등학교', '매홀초등학교', '아이펀 어린이집', '시립오산어린이집', '오르다유치원', '화성초등학교', '창의나라 유치원', '대일유치원', '하얀뜰어린이집', '양산초등학교', '지예뜰유치원', '노틀담유치원', '로뎀장애아전담 어린이집', '시립비둘기어린이집', '둥근해어린이집', '지움 어린이집', '광성초등학교', '아이사랑어린이집', '청원어린이집', '오산고현초등학교', '대호초등학교', '별새꽃유치원', '세미초등학교', '시립남부어린이집', '키즈청호어린이집', '문시초등학교', '시립생명숲어린이집']"

#### (참고) 결과검증용: 위험지역(시립은여울어린이집) vs 비위험지역(해오름어린이집) 비교
- 위험지수가 높은 시립은여울어린이집의 경우, 사고유무에 영향을 미치는 독립변수들의 값이 다른 지역들에 비해 높음을 확인

In [109]:
""" 분석 이후 결과의 유의성을 검증하기 위한 용도로 나중에 사용 (PPT 분석결과 부분에 첨부)"""

temp_list = ['시립은여울어린이집', '해오름어린이집']
temp = data_norm_sorted.copy()
temp = temp.loc[temp['gid'].isin(temp_list)]
temp

,혼잡빈도강도합,유동인구_14시_19시,주정차단속,아파트수,승용차_14시_19시,사거리,score,gid
88,0.250536,0.351852,1.000000,0.245902,0.340902,1.0,1.411401,시립은여울어린이집
74,0.272533,0.018519,0.004079,0.098361,0.160614,0.0,0.335456,해오름어린이집


# 4. 시설물 우선설치지역 선정
- 설명: 비효율 지역이면서, 동시에 사고위험도 높은 지역을 찾는다

## 4-1. 비효율, 사고위험 두 조건을 모두 만족하는 지역 선정 (23곳)
- 설명: 효율성점수 상위41곳 (효율성점수가 높을수록 비효율적) 과 교통사고 위험지수 상위40곳의 교집합으로 23곳을 골라 낸다

In [110]:
""" 효율성점수 상위41곳, 교통사고 위험지수 상위40곳 리스트 각각 저장 """

eff_list = list(dea_eff_41.schoolzone)
score_list = list(data_norm_sorted.gid[:40])


""" 상위41곳&상위40곳 교집합으로 선정된 23개 """

def intersection(lst1, lst2): 
    return list(set(lst1) & set(lst2)) 

name_list = intersection(eff_list, score_list)
print(len(name_list))

23


In [111]:
str(name_list)

"['창의나라 유치원', '오산원일초등학교', '물향기 유치원', '동화마을유치원', '시립매홀어린이집', '화성초등학교', '시립은여울어린이집', '오산대원초등학교', '원동초등학교', '시립남부어린이집', '지예뜰유치원', '운산초등학교', '로뎀장애아전담 어린이집', '오산고현초등학교', '시립비둘기어린이집', '별새꽃유치원', '지움 어린이집', '시립오산어린이집', '대호초등학교', '매홀초등학교', '운천초등학교', '대일유치원', '아이펀 어린이집']"

## 4-2. 시각화를 통한 중복격자 제거로 최종 지역 선정 (20곳)
- 설명: 위에서 구한 23곳 중, 어린이보호구역이 서로 가까워서 격자가 거의 겹치는 곳을 찾아내 3곳의 지역을 제외하여 최종 20곳을 선정한다

In [112]:
df_temp = df_grid.copy()

df_temp['geometry'] = None

lon_1m = 1/88.74/1000
lat_1m = 1/109.96/1000  
rng = 200

for i in range(len(df_temp)):
    f = []
    a = (df_temp['보호구역_경도'][i]+lon_1m*rng,df_temp['보호구역_위도'][i]+lat_1m*rng)
    b = (df_temp['보호구역_경도'][i]+lon_1m*rng,df_temp['보호구역_위도'][i]-lat_1m*rng)
    c = (df_temp['보호구역_경도'][i]-lon_1m*rng,df_temp['보호구역_위도'][i]-lat_1m*rng)
    d = (df_temp['보호구역_경도'][i]-lon_1m*rng,df_temp['보호구역_위도'][i]+lat_1m*rng)
    f.append(a);f.append(b);f.append(c);f.append(d)
    f_poly = Polygon(f)
    df_temp['geometry'][i] = f_poly
    
df_grid00 = df_temp.loc[df_temp['gid'].isin(name_list)]
df_grid00.shape

# Make layer
##
layer1 = pdk.Layer( 'PolygonLayer', # 사용할 Layer 타입 
                  df_grid00, # 시각화에 쓰일 데이터프레임 
                  get_polygon='coordinates', # geometry 정보를 담고있는 컬럼 이름 
                  get_fill_color='[255, 0, 0, 100 ]', # 각 데이터 별 rgb 또는 rgba 값 (0~255)
                  pickable=True, # 지도와 interactive 한 동작 on 
                  auto_highlight=True # 마우스 오버(hover) 시 박스 출력 
                 ) 

# Set the viewport location 
center = [127.05057, 37.16336] 
view_state = pdk.ViewState( 
    longitude=center[0], 
    latitude=center[1], 
    zoom=12
) 

# Render 
r = pdk.Deck(layers=[ layer1], initial_view_state=view_state,
             map_style='mapbox://styles/mapbox/outdoors-v11',
             mapbox_key = "pk.eyJ1IjoiamNsYXJhODExIiwiYSI6ImNrZzF4bWNhdTBpNnEydG54dGpxNDEwajAifQ.XWxOKQ-2HqFBVBYa-XoS-g"
            )

r.to_html()

- 지도 아래쪽에서 겹치는 격자를 찾아내 [시립남부어린이집, 대일유치원, 별새꽃유치원] 세 곳은 제외해도 다른 격자가 그 지역을 커버하는 것을 확인

In [113]:
""" 23곳 중 중복 제거해서 최종 20곳 선정 """

name_list.remove('시립남부어린이집')
name_list.remove('대일유치원')
name_list.remove('별새꽃유치원')
print(len(name_list))

20


# 5. 추가설치 시설물 종류 및 개수 파악
- 설명: 최종 선정한 20곳을 대상으로 벤치마킹 분석을 실행해, 추가 시설물 종류와 개수를 파악한다

In [114]:
""" y값에만 관심있음 """

dea_data = dea_data.loc[:, ~dea_data.columns.str.startswith('x')]
dea_data = dea_data.drop(['y5_신호등'], axis=1) # 신호등 변수는 분석에서 제외하기로 함 (R에서 효율성 점수 계산할 때 제외함)
dea_data.rename(columns = {'y6_CCTV_과속단속' : 'y5_CCTV_과속단속'}, inplace = True)
print(dea_data.shape)
dea_data.sample()

(91, 6)


,보호구역명,y1_CCTV_보호구역,y2_단속카메라_불법주정차,y3_도로안전표지,y4_과속방지턱,y5_CCTV_과속단속
22,필봉초등학교,1,0,3,3,0


## 5-1. 추가설치 시설물 목표치 계산
- 설명: DEA 분석에서 제공되는 참조집단의 가중치를 활용하여, 각 비효율 지역을 대상으로 효율성을 확보하기 위한 산출물 목표치를 계산한다 (구체적인 계산 수식은 분석 보고서에 작성)

In [115]:
"""
벤치마킹 위한 함수 정의
df: DEA 데이터 (보호구역명이랑 y 값으로만 이루어진 데이터프레임)
df_lambda: R에서 실행 후 가져온 참조집단 가중치 <- lambda(result)
"""

import numpy as np
def get_benchmarking(df, df_lambda): 
    
    """ 참조집단의 가중치 """
    weight_dict = {}
    for index,row in df_lambda.iterrows():
        temp = list(row) # 리스트: 가중치 숫자들
        temp.pop(0)
        weight_dict[row['schoolzone']] = temp
        
    """ 참조집단 리스트 """
    reference_set = list(df_lambda.columns) # 리스트: 참조집단 이름들
    reference_set.pop(0) 
    
    """ 참조집단 데이터 """
    data_dict = {}
    for index,row in df.iterrows():
        temp = list(row) # 리스트: 데이터 숫자들
        temp.pop(0)
        data_dict[row['보호구역명']] = temp
    
    """ 참조집단 가중치 데이터로 산출물 목표치 계산 """
    dmu_set = list(df_lambda.schoolzone)
    new_data_dict = {}
    weighted_data_dict = {}

    for dmu in dmu_set:
        weight = weight_dict[dmu]
        for i in range(len(reference_set)):
            temp = data_dict[reference_set[i]] # 참조대상의 데이터
            temp = np.array(temp) 
            new_data = temp*weight[i] # 참조대상 데이터 * 가중치
            new_data_dict[reference_set[i]] = new_data     
        weighted_data_dict[dmu] = pd.DataFrame.from_dict(new_data_dict, orient='index',
                                                        columns=['y1_CCTV_보호구역','y2_단속카메라_불법주정차',
                                                                 'y3_도로안전표지','y4_과속방지턱',
                                                                 'y5_CCTV_과속단속'] ) 
    """ 필요한 정보 """
    final_dict = {}
    for dmu in dmu_set:
        final_dict[dmu] = dict(weighted_data_dict[dmu].sum())

    return final_dict

In [116]:
""" 예시) 삼미초등학교의 시설물 목표치 계산 """

result = get_benchmarking(dea_data, dea_lambda_crs)
result['삼미초등학교']

{'y1_CCTV_보호구역': 2.0,
 'y2_단속카메라_불법주정차': 1.0,
 'y3_도로안전표지': 1.0,
 'y4_과속방지턱': 0.0,
 'y5_CCTV_과속단속': 0.0}

## 5-2. 추가설치 시설물 목표치 보기
- 설명: 최종 선정된 20곳을 대상으로, 시설물(산출물) 현재 개수, 목표 개수, 개선해야 할 개수를 나타낸다

In [117]:
""" 
최종 20곳 시설물 목표치 보기 위한 함수 정의
name_list: 최종 20곳의 이름 리스트
"""

def compare(name_list):
    
    result_dict = {}

    for name in name_list:
    
        original = list(dea_data[dea_data['보호구역명'] == name].iloc[0])
        original.pop(0)
        original = np.array(original)
        result_dict[name+'_현재'] = original.tolist()

        goal = list(result[name].values())
        goal = np.array(goal)
        result_dict[name+'_목표'] = goal.tolist()

        sub = goal - original
        result_dict[name+'_개선'] = sub.tolist()
        
        percent = sub/original*100
        result_dict[name+'_개선%'] = percent.tolist()
    
    result_df = pd.DataFrame.from_dict(result_dict, orient='index', 
                                       columns=['y1_CCTV_보호구역', 'y2_단속카메라_불법주정차', 'y3_도로안전표지', 
                                                'y4_과속방지턱', 'y5_CCTV_과속단속'])
    return result_df


""" 최종 선정 20곳 리스트"""
print(len(name_list))

benchmarking_result = compare(name_list)
pd.set_option('display.max_columns', 500)
benchmarking_result.T

20


,창의나라 유치원_현재,창의나라 유치원_목표,창의나라 유치원_개선,창의나라 유치원_개선%,오산원일초등학교_현재,오산원일초등학교_목표,오산원일초등학교_개선,오산원일초등학교_개선%,물향기 유치원_현재,물향기 유치원_목표,물향기 유치원_개선,물향기 유치원_개선%,동화마을유치원_현재,동화마을유치원_목표,동화마을유치원_개선,동화마을유치원_개선%,시립매홀어린이집_현재,시립매홀어린이집_목표,시립매홀어린이집_개선,시립매홀어린이집_개선%,화성초등학교_현재,화성초등학교_목표,화성초등학교_개선,화성초등학교_개선%,시립은여울어린이집_현재,시립은여울어린이집_목표,시립은여울어린이집_개선,시립은여울어린이집_개선%,오산대원초등학교_현재,오산대원초등학교_목표,오산대원초등학교_개선,오산대원초등학교_개선%,원동초등학교_현재,원동초등학교_목표,원동초등학교_개선,원동초등학교_개선%,지예뜰유치원_현재,지예뜰유치원_목표,지예뜰유치원_개선,지예뜰유치원_개선%,운산초등학교_현재,운산초등학교_목표,운산초등학교_개선,운산초등학교_개선%,로뎀장애아전담 어린이집_현재,로뎀장애아전담 어린이집_목표,로뎀장애아전담 어린이집_개선,로뎀장애아전담 어린이집_개선%,오산고현초등학교_현재,오산고현초등학교_목표,오산고현초등학교_개선,오산고현초등학교_개선%,시립비둘기어린이집_현재,시립비둘기어린이집_목표,시립비둘기어린이집_개선,시립비둘기어린이집_개선%,지움 어린이집_현재,지움 어린이집_목표,지움 어린이집_개선,지움 어린이집_개선%,시립오산어린이집_현재,시립오산어린이집_목표,시립오산어린이집_개선,시립오산어린이집_개선%,대호초등학교_현재,대호초등학교_목표,대호초등학교_개선,대호초등학교_개선%,매홀초등학교_현재,매홀초등학교_목표,매홀초등학교_개선,매홀초등학교_개선%,운천초등학교_현재,운천초등학교_목표,운천초등학교_개선,운천초등학교_개선%,아이펀 어린이집_현재,아이펀 어린이집_목표,아이펀 어린이집_개선,아이펀 어린이집_개선%
y1_CCTV_보호구역,1.0,1.474599,0.474599,47.459906,2.0,8.688297,6.688297,334.414848,1.0,2.850640,1.850640,185.064017,1.0,5.082001,4.082001,408.200073,1.0,6.823056,5.823056,582.305601,3.0,3.712220,0.712220,23.740655,0.0,5.340061,5.340061,inf,2.0,10.042996,8.042996,402.149802,2.0,4.530469,2.530469,126.523468,1.0,2.703459,1.703459,170.345913,2.0,10.067338,8.067338,403.366877,1.0,4.459315,3.459315,345.931498,2.0,3.856348,1.856348,92.817417,1.0,1.938524,0.938524,93.852391,1.0,3.299107,2.299107,229.910697,1.0,6.605119,5.605119,560.511948,3.0,7.612736,4.612736,153.757880,2.0,8.612289,6.612289,330.614458,1.0,6.405976,5.405976,540.597603,1.0,2.168959,1.168959,116.895901
y2_단속카메라_불법주정차,0.0,0.000000,0.000000,NaN,2.0,8.518271,6.518271,325.913564,1.0,2.124392,1.124392,112.439182,1.0,1.617647,0.617647,61.764706,1.0,0.946323,-0.053677,-5.367656,2.0,2.207055,0.207055,10.352750,2.0,5.157895,3.157895,157.894737,2.0,7.103947,5.103947,255.197349,0.0,2.130867,2.130867,inf,0.0,1.113119,1.113119,inf,1.0,1.488507,0.488507,48.850708,1.0,1.481962,0.481962,48.196178,0.0,0.198172,0.198172,inf,0.0,0.000000,0.000000,NaN,0.0,0.000000,0.000000,NaN,3.0,4.944220,1.944220,64.807345,1.0,1.976357,0.976357,97.635718,2.0,2.983130,0.983130,49.156498,1.0,5.260508,4.260508,426.050838,0.0,2.605381,2.605381,inf
y3_도로안전표지,2.0,2.949617,0.949617,47.480834,3.0,12.862420,9.862420,328.747325,6.0,8.792530,2.792530,46.542173,2.0,3.670299,1.670299,83.514938,6.0,9.039550,3.039550,50.659168,6.0,7.469249,1.469249,24.487481,10.0,13.024271,3.024271,30.242712,5.0,18.632282,13.632282,272.645640,7.0,14.021288,7.021288,100.304115,3.0,5.837830,2.837830,94.594341,7.0,18.169210,11.169210,159.560150,4.0,6.826663,2.826663,70.666584,4.0,7.457932,3.457932,86.448290,0.0,0.000000,0.000000,NaN,3.0,9.054629,6.054629,201.820980,2.0,5.832661,3.832661,191.633053,2.0,4.525549,2.525549,126.277445,2.0,2.979439,0.979439,48.971926,7.0,7.431143,0.431143,6.159183,0.0,3.576250,3.576250,inf
y4_과속방지턱,0.0,0.000000,0.000000,NaN,0.0,0.000000,0.000000,NaN,1.0,1.291869,0.291869,29.186939,0.0,0.000000,0.000000,NaN,2.0,2.589199,0.589199,29.459964,0.0,0.493348,0.493348,inf,0.0,0.000000,0.000000,NaN,1.0,3.275972,2.275972,227.597248,0.0,4.616383,4.616383,inf,0.0,0.000000,0.000000,NaN,0.0,0.000000,0.000000,NaN,0.0,0.019725,0.019725,inf,0.0,0.000000,0.000000,NaN,0.0,0.000000,0.000000,NaN,0.0,0.000000,0.000000,NaN,0.0,0.000000,0.000000,NaN,0.0,0.000000,0.000000,NaN,1.0,1.042979,0.042979,4.297940,1.0,0.752235,-0.247765,-24.776520,0.0,0.000000,0.000000,NaN
y5_CCTV_과속단속,0.0,0.000000,0.000000,NaN,0.0,0.000000,0.000000,NaN,0.0,0.000000,0.000000,NaN,0.0,0.000000,0.000000,NaN,0.0,0.000000,0.000000,NaN,0.0,0.000000,0.000000,NaN,0.0,0.000000,0.000000,NaN,0.0,0.000000,0.000000,NaN,0.0,0.000000,0.000000,NaN,0.0,0.000000,0.000000,NaN,0.0,0.000000,0.000000,NaN,0.0,0.000000,0.000000,NaN,0.0,0.000000,0.000000,NaN,0.0,0.000000,0.000000,NaN,0.0,0.000000,0.000000,NaN,0.0,0.000000,0.000000,NaN,0.0,0.000000,0.000000,NaN,0.0,0.000000,0.000000,NaN,0.0,0.000000,0.000000,NaN,0.0,0.000000,0.000000,NaN


- 현재: 현재 시설물 개수
- 목표: 효율성을 확보하기 위한 시설물 개수의 목표치
- 개선: (현재 개수) - (목표 개수)
- 개선%: {(현재 개수) - (목표 개수)}/(현재 개수) * 100

# (참고) 효율성점수, 교통사고위험지수 결과물 시각화
- 설명: 어린이보호구역 91곳을 대상으로 점수를 지도에 시각화
- 색깔이 밝을수록 점수가 높은 것 (= 비효율적, 사고위험 높음)

In [118]:
""" 결과물 시각화를 위한 함수 정의 """
def result_grid(data, val):

    # val 열 na 제거
    data[val] = data[val].fillna(0)

    # 정규화
    data['val_정규화'] = data[val] / data[val].max()

    # geometry를 coordinate 형태로 적용
    #data['coordinates'] = data['geometry'].apply(multipolygon_to_coordinates) #pydeck 을 위한 coordinate type

    # Make layer
    layer = pdk.Layer( 'PolygonLayer', # 사용할 Layer 타입 
                      data[(data[val].isnull()==False) & data[val]!=0], # 시각화에 쓰일 데이터프레임 
                      get_polygon='coordinates', # geometry 정보를 담고있는 컬럼 이름 
                      get_fill_color='[0, 255*val_정규화, 0, val_정규화*1000 ]', # 각 데이터 별 rgb 또는 rgba 값 (0~255)
                      pickable=True, # 지도와 interactive 한 동작 on 
                      auto_highlight=True # 마우스 오버(hover) 시 박스 출력 
                     ) 

    # Set the viewport location 
    center = [127.05057, 37.16336] 
    view_state = pdk.ViewState( 
        longitude=center[0], 
        latitude=center[1], 
        zoom=12
    ) 

    # Render 
    r = pdk.Deck(layers=[layer], initial_view_state=view_state,
                 map_style='mapbox://styles/mapbox/outdoors-v11',
                 mapbox_key = "pk.eyJ1IjoiamNsYXJhODExIiwiYSI6ImNrZzF4bWNhdTBpNnEydG54dGpxNDEwajAifQ.XWxOKQ-2HqFBVBYa-XoS-g"
                )

    return r.to_html()

In [119]:
""" 효율성점수 시각화 """

dea_eff_ = dea_eff.rename(columns = {'schoolzone' : 'gid'}, inplace = False)
df_dea_eff = pd.merge(df_grid, dea_eff_, on="gid")

df_dea_eff['geometry'] = None

lon_1m = 1/88.74/1000
lat_1m = 1/109.96/1000  
rng = 200

for i in range(len(df_dea_eff)):
    f = []
    a = (df_dea_eff['보호구역_경도'][i]+lon_1m*rng,df_dea_eff['보호구역_위도'][i]+lat_1m*rng)
    b = (df_dea_eff['보호구역_경도'][i]+lon_1m*rng,df_dea_eff['보호구역_위도'][i]-lat_1m*rng)
    c = (df_dea_eff['보호구역_경도'][i]-lon_1m*rng,df_dea_eff['보호구역_위도'][i]-lat_1m*rng)
    d = (df_dea_eff['보호구역_경도'][i]-lon_1m*rng,df_dea_eff['보호구역_위도'][i]+lat_1m*rng)
    f.append(a);f.append(b);f.append(c);f.append(d)
    f_poly = Polygon(f)
    df_dea_eff['geometry'][i] = f_poly
    

result_grid(df_dea_eff, 'crs_eff')

- 색깔이 밝을수록 점수가 높은 것 (= 비효율적, 즉 투입요소에 비해 시설물이 부족함)

In [120]:
""" 교통사고위험지수 시각화 """

df_data_norm_sorted = pd.merge(df_grid, data_norm_sorted, on="gid")
df_data_norm_sorted['geometry'] = None

lon_1m = 1/88.74/1000
lat_1m = 1/109.96/1000  
rng = 200

for i in range(len(df_dea_eff)):
    f = []
    a = (df_data_norm_sorted['보호구역_경도'][i]+lon_1m*rng,df_data_norm_sorted['보호구역_위도'][i]+lat_1m*rng)
    b = (df_data_norm_sorted['보호구역_경도'][i]+lon_1m*rng,df_data_norm_sorted['보호구역_위도'][i]-lat_1m*rng)
    c = (df_data_norm_sorted['보호구역_경도'][i]-lon_1m*rng,df_data_norm_sorted['보호구역_위도'][i]-lat_1m*rng)
    d = (df_data_norm_sorted['보호구역_경도'][i]-lon_1m*rng,df_data_norm_sorted['보호구역_위도'][i]+lat_1m*rng)
    f.append(a);f.append(b);f.append(c);f.append(d)
    f_poly = Polygon(f)
    df_data_norm_sorted['geometry'][i] = f_poly
    

result_grid(df_data_norm_sorted, 'score')

- 색깔이 밝을수록 점수가 높은 것 (= 사고위험 높음)

# (참고) 최종선정 20곳 순위 매기기: 효율성 점수 순
- 효율성 점수: 높을수록 비효율적임을 의미
- 최종선정된 20곳을 비효율적인 순서로 정렬해 순위를 매김
- 아래 데이터프레임을 엑셀로 옮겨서 최종결과물로 제출

In [121]:
final = name_list

df_final = df_grid.loc[df_grid['gid'].isin(final)]
print(df_final.shape)
df_final = df_final[['gid', '보호구역_경도', '보호구역_위도']]

dea_eff_crs = dea_eff[['schoolzone', 'crs_eff']]
dea_eff_crs.rename(columns = {'schoolzone': 'gid'}, inplace = True)

df_final_sorted = pd.merge(df_final, dea_eff_crs, on="gid", how='left')
df_final_sorted = df_final_sorted.sort_values('crs_eff', ascending=False)

df_final_sorted

(20, 31)


,gid,보호구역_경도,보호구역_위도,crs_eff
4,오산원일초등학교,127.071695,37.139203,4.344148
3,오산대원초등학교,127.071191,37.130553,3.761123
17,지움 어린이집,127.052993,37.139836,3.048369
5,운산초등학교,127.077534,37.152640,2.649629
0,대호초등학교,127.057889,37.159106,2.537579
18,아이펀 어린이집,127.049790,37.140137,2.284425
9,동화마을유치원,127.076416,37.157039,2.049449
7,원동초등학교,127.069748,37.135528,2.003041
16,시립비둘기어린이집,127.072211,37.133451,1.973532
12,지예뜰유치원,127.083871,37.137570,1.949832
